<head><h1 align="center">
Food.com Recipe Interactions
</h1></head>  
  
<head><h3 align="center">Recipe Recommender System with Spark</h3></head>

#### Setup

In [10]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request, time, csv, json, pickle
from os import system
from math import floor
from copy import deepcopy
from time import gmtime, strftime  
import numpy as np
import pandas as pd                   
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cdist

from IPython.display import Image  
from IPython.display import display  
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer  
import pyspark  
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, IntegerType
# Additional import statements are my own, incase I would like to copy the above tutorial statement to other notebook.
pd.set_option('display.max_rows', 6)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 200)

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/knn'
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/knn:1',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/knn:1',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/knn:1',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/knn:1',
              'ap-northeast-1': '351501993468.dkr.ecr.ap-northeast-1.amazonaws.com/knn:1',
              'ap-northeast-2': '835164637446.dkr.ecr.ap-northeast-2.amazonaws.com/knn:1',
              'ap-southeast-2': '712309505854.dkr.ecr.ap-southeast-2.amazonaws.com/knn:1'}

my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

bucket = 'sagemaker-studio-t1ems8mtnoj'
subfolder = ''
s3 = boto3.client('s3')
contents = s3.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
# for f in contents:
#     print(f['Key'])

Success - the MySageMakerInstance is in the us-east-2 region. You will use the 404615174143.dkr.ecr.us-east-2.amazonaws.com/knn:1 container for your SageMaker endpoint.


# Additional Data Import & Preparation

In [12]:
# My Bucket ARN Access Key, just in case
arn = 'arn:aws:s3:us-east-2:133716175259:accesspoint/recipe-book-GitHub-t2-exec-to-s3'

In [12]:
spark = SparkSession.builder.appName('Recipe Book').getOrCreate()

In [ ]:
df = pd.read_csv('../data/ui_ratings.csv',index_col=0)

In [16]:
# try:
#     urllib.request.urlretrieve ('https://sagemaker-studio-t1ems8mtnoj.s3.us-east-2.amazonaws.com/ui_ratings.csv', 'data/ui_ratings.csv')
#     print('Success: downloaded ui_ratings.csv.')
# except Exception as e:
#     print('Data load error: ',e)

# try:
#     df = pd.read_csv('data/ui_ratings.csv',index_col=0)
#     print('Success: Data loaded into dataframe.')
# except Exception as e:
#     print('Data load error: ',e)

Data load error:  HTTP Error 403: Forbidden
Success: Data loaded into dataframe.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
df

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5
...,...,...,...
1132362,116593,72730,0
1132363,583662,386618,5
1132364,157126,78003,5
1132365,53932,78003,4


In [37]:
# rdf = pd.read_csv('../data/RAW_recipes.csv')
# idf = pd.read_csv('../data/RAW_interactions.csv')

# pp_rdf = pd.read_csv('../data/PP_recipes.csv')
# pp_idf = pd.read_csv('../data/PP_users.csv')

# ingID = pd.read_pickle('../data/ingr_map.pkl')

# tdf = pd.read_csv('../data/interactions_train.csv')
# vdf = pd.read_csv('../data/interactions_validation.csv')

# # """Cleaning: All steps carried over from EDA Notebook"""
# rdf.drop(labels=721, inplace = True)

# # Cleaning/FE: Creating columns for recipe's respective nutrients
# rdf['kcal'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[0])
# rdf['fat'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[1])
# rdf['sugar'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[2])
# rdf['salt'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[3])
# rdf['protein'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[4])
# rdf['sat_fat'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[5])
# rdf['carbs'] = rdf.nutrition.apply(lambda x: x[1:-1].split(sep=', ')[6])

# # Cleaning: Imputing outlier value to median
# rdf['minutes'] = np.where(rdf.minutes == 2147483647,
#                          rdf.minutes.median(),
#                          rdf.minutes)

# idf['date'] = pd.to_datetime(idf.date)

In [18]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [19]:
schema = StructType([ \
    StructField("user_id", IntegerType(), True), \
    StructField("recipe_id", IntegerType(), True), \
    StructField("rating", IntegerType(), True)])

In [39]:
# SparkSession.createDataFrame(df, data='data/ui_ratings.csv', schema=schema)

In [24]:
df

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5
...,...,...,...
1132362,116593,72730,0
1132363,583662,386618,5
1132364,157126,78003,5
1132365,53932,78003,4


In [29]:
ui = spark.createDataFrame(df)

In [30]:
# rdf = spark.read.csv('../data/RAW_recipes.csv', sep=",", quote="", header=True, multiLine=True, inferSchema=True).\
#     option("escape", '\\').option("escape", ':').option("escape", '\n')

# idf = spark.read.csv('../data/RAW_interactions.csv', sep=",", quote="", header=True, multiLine=True, inferSchema=True).\
#     option("escape", '\\').option("escape", ':').option("escape", '\n')

# pp_rdf = spark.read.csv('../data/PP_recipes.csv', sep=",", quote="", header=True, multiLine=True, inferSchema=True).\
#     option("escape", '\\').option("escape", ':').option("escape", '\n')

# pp_idf = spark.read.csv('../data/PP_users.csv', sep=",", quote="", header=True, multiLine=True, inferSchema=True).\
#     option("escape", '\\').option("escape", ':').option("escape", '\n')

# ingID = pd.read_pickle('../data/ingr_map.pkl')

# tdf = spark.read.csv('../data/interactions_train.csv', sep=",", quote="", header=True, multiLine=True, inferSchema=True).\
#     option("escape", '\\').option("escape", ':').option("escape", '\n')

# vdf = spark.read.csv('../data/interactions_validation.csv', sep=",", quote="", header=True, multiLine=True, inferSchema=True).\
#     option("escape", '\\').option("escape", ':').option("escape", '\n')

# Spark

## Model-based

In [31]:
ui.show()

+----------+---------+------+
|   user_id|recipe_id|rating|
+----------+---------+------+
|     38094|    40893|     4|
|   1293707|    40893|     5|
|      8937|    44394|     4|
|    126440|    85009|     5|
|     57222|    85009|     5|
|     52282|   120345|     4|
|    124416|   120345|     0|
|2000192946|   120345|     2|
|     76535|   134728|     4|
|    273745|   134728|     5|
|    353911|   134728|     5|
|    190375|   134728|     5|
|    468945|   134728|     0|
|    255338|   134728|     5|
|   1171894|   134728|     5|
|    136726|   197160|     5|
|     68960|   200236|     4|
|    618928|   200236|     4|
|    217118|   200236|     5|
|2000049093|   200236|     5|
+----------+---------+------+
only showing top 20 rows



In [32]:
ui.dtypes

[('user_id', 'bigint'), ('recipe_id', 'bigint'), ('rating', 'bigint')]

In [33]:
ui.describe() # describe says strings?

DataFrame[summary: string, user_id: string, recipe_id: string, rating: string]

In [34]:
ui.dtypes

[('user_id', 'bigint'), ('recipe_id', 'bigint'), ('rating', 'bigint')]

In [38]:
type(ui)

pyspark.sql.dataframe.DataFrame

## Memory-based

In [49]:
trainset, testset = train_test_split(sidf, test_size = 0.2, random_state = 1984)

In [50]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  192093 

Number of items:  211233 



In [53]:
sim_cos = {'name':'cosine', 'user_based':True}

In [54]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 332. GiB for an array with shape (211233, 211233) and data type float64

### Collaborative Filtering — User-based Recommender System

>- Collaborative Filtering (CF) is currently the most widely used approach to build recommendation systems  
>- The key idea behind CF is that similar users have similar interests and that a user generally likes items that are similar to other items they like  
>- CF is filling an "empty cell" in the utility matrix based on the similarity between users or item. Matrix factorization or decomposition can help us solve this problem by determining what the overall "topics" are when a matrix is factored
  
Note: We will likely need to bring this notebook over to DataBricks or AWS.  
  
We can use **cosine similarity** between users, or **Pearson Correlation Coefficient**. Different metrics will offer different results, but I don't see why we couldn't offer multiple recommendations using different metrics.

**Note: Don't forget to downplay zeros as they are gaps, not values representative of sentiment!**

### SVD — Singular Value (Matrix) Decomposition

Notes:  
- eigendecompositions require square matrices
- for SVD, we can create square matrices by taking the dot product of a matrix and its transpose
- we can use SVD to draw vectors in a new space to capture as much of the variance in our data as possible

~~Below we use the raw interactions dataset to create the rating matrix `A`, with rows representing recipes and columns representing users.~~  
NVM LOOKS LIKE WE'RE GONNA NEED DISTRIBUTED COMPUTING FOR EVEN A SIMPLE SVD



In [14]:
A = np.ndarray(
    shape=(np.max(idf.recipe_id.values), np.max(idf.index.values)),
    dtype=np.uint8)

A[idf.recipe_id.values-1, idf.index.values-1] = idf.rating.values

MemoryError: Unable to allocate 567. GiB for an array with shape (537716, 1132366) and data type uint8

### ALS — Alternating Lease Squares Matrix Decomposition

- plug in guess values for P and Q
- hold the values of one constant, then use the values for R and the non-constant to find the optimum values for.
- repeat for the other

> "When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

For collaborative ALS, we will want our data to be shaped something like below, where the column numbers represent different recipes.  

Recipe->| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 |...| n  
:-------|:--|---|---|---|---|---|---|---|---|---|---|---|---|---|---:  
user_01 | 0 | 0 | 5 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  
user_02 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_03 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 5 | 0 | 0  
user_04 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_05 | 0 | 0 | 0 | 2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  
user_06 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0  
user_07 | 0 | 0 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 5 | 0 | 0 | 0 | 0  
  ...   
user_n  | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 5 | 0 | 0 | 0  


# STOP SPARK 

In [40]:
spark.stop()

# Evaluation

# Reference

## Reading in Big Data from S3

## Notes

Reference:  
>(*Both personalized and content-based recommendation systems*) make use of different similarity metrics to determine how "similar" items are to one another. The most common similarity metrics are [**Euclidean distance**](https://en.wikipedia.org/wiki/Euclidean_distance), [**cosine similarity**](https://en.wikipedia.org/wiki/Cosine_similarity), [**Pearson correlation**](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and the [**Jaccard index**](https://en.wikipedia.org/wiki/Jaccard_index) (useful with binary data). Each one of these distance metrics has its advantages and disadvantages depending on the type of ratings you are using and the characteristics of your data.$_1$  
  

## Further Reading

For those of you visiting this page who are interested in reading more about recommender systems, below are fantastic resources that I have collected.  
$_n$$_o$$_w$ &nbsp;$_I$ &nbsp;$_a$$_m$ &nbsp;$_t$$_h$$_e$ &nbsp;$_r$$_e$$_c$$_o$$_m$$_m$$_e$$_n$$_d$$_a$$_t$$_i$$_o$$_n$ &nbsp;$_s$$_y$$_s$$_t$$_e$$_m$$_!$

[*Mining Massive Datasets: Chapter 9*](http://infolab.stanford.edu/~ullman/mmds/ch9.pdf), © Copyright Stanford University. Stanford, California 94305  
[Singular Value Decomposition (SVD) & Its Application In Recommender System](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/), Dr. Vaibhav Kumar

# Bibliography

1. [*Introduction to Recommender Systems* by Flatiron School](https://github.com/learn-co-curriculum/dsc-recommendation-system-introduction) is licensed under CC BY-NC-SA 4.0
>© 2018 Flatiron School, Inc.  
>© 2021 Flatiron School, LLC

# Notes

In [11]:
contents[0]

{'Key': 'RAW_interactions.csv',
 'LastModified': datetime.datetime(2021, 4, 29, 17, 32, 1, tzinfo=tzlocal()),
 'ETag': '"b76f667f757a6e0b6d3b08d23e0bc5e0-21"',
 'Size': 349436524,
 'StorageClass': 'STANDARD',
 'Owner': {'ID': 'ee74dd80de8b382fe7543b22cf2221ca7e68c76f28d55bec19b96bbe32873a4c'}}

In [12]:
contents[0]['Key']

'RAW_interactions.csv'

In [ ]:
# import boto3
# import sys

# if sys.version_info[0] < 3: 
#     from StringIO import StringIO # Python 2.x
# else:
#     from io import StringIO # Python 3.x

# # get your credentials from environment variables
# aws_id = os.environ['133716175259']
# aws_secret = os.environ['AWS_SECRET']

# client = boto3.client('s3', aws_access_key_id=aws_id,
#         aws_secret_access_key=aws_secret)

# bucket_name = 'my_bucket'

# object_key = 'my_file.csv'
# csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
# body = csv_obj['Body']
# csv_string = body.read().decode('utf-8')

# df = pd.read_csv(StringIO(csv_string))